In [ ]:
# Importing relevant libraries
from yahoo_fin.stock_info import tickers_dow
from scipy.stats as stats
import timeit
import pandas as pd
from yfinance as yf
import numpy as np

asset_class = ['equity', 'commodity', 'IR', 'Fx']
max_weights = [0.25, 0.25, 0.25, 0.25]

In [ ]:
# Data extraction for all the asset class
extracted_data = yf.download(asset_list, start = '2010-01-01', end = '2020-01-01')
data = extracted_data['Adj_Close'].dropna()
returns = data.pct_change(1)
returns = returns[asset_list].dropna()

In [ ]:
# Variance-Covariance VaR Method
def parametric_var(returns, weights, conf_int):
    portfolio_returns = np.sum(weights*returns, axis=1)
    mean_returns = portfolio_returns.mean()
    std_returns = portfolio_returns.std()
    zvalue = stats.norm.ppf(conf_int)
    par_VaR = mean_returns + zvalue*std_returns
    return par_VaR

In [ ]:
# Monte Carlo Simulation VaR Method
def monte_carlo_var(returns, weights, conf_int, sims=1000):
    corr_mat = returns.corr()
    L = np.linalg.cholesky(corr_mat)
    mean_returns = returns.mean()
    std_returns = returns.std()
    portfolio_returns= []
    for i in range(sims):
        z_value = np.random.randn(1, len(weights))
        z_dash = np.transpose(np.matmul(L, np.transpose(z_values)))
        return_run = returns.mean().values + returns.std().values.z_dash
        portfolio_returns_run = np.sum(weights*return_run, axis=1)
        portfolio_returns.append(portfolio_returns_run)
    MCS_VaR = np.percentile(portfolio_returns, 1-conf_int)
    return MCS_VaR
        

In [ ]:
# Historical Simulation VaR method
def historical_simulation_var(returns, weights, conf_int):
    portfolio_returns = np.sum(weights*returns, axis=1)
    historic_var = -np.percentile(portfolio_returns, (1-conf_int)*100)
    return historic_var

In [ ]:
# VaR calculation comparison
def var_calc_comparison(returns, weights, conf_int, sims=1000):
    start_time = timeit.default_timer()
    vars_cal = []
    vars_cal.append(parametric_var(returns, weights, 0.99))
    t1 = timeit.default_timer()
    vars_cal.append(historical_simulation_var(returns, weights, 0.99))
    t2 = timeit.default_timer()
    vars_cal.append(monte_carlo_var(returns, weights, 0.99, 1000))
    t3 = timeit.default_timer()
    times = [t1-start_time, t2-start_time, t3-start_time]


In [ ]:
# Weights optimization
def weights_optimization(returns, max_weights, target_var, conf_inf, no_of_sims):
    ret_var = 0.0
    p = 0
    while(p<=no_of_sims):
        wts = np.random.random(len(asset_class))
        wts = wts/np.sum(wts)
        if (wts<=np.array(max_weights)).all():
            port_returns = np.sum(wts.returns, axis=1)
            total_return = ((1+port_returns).cumprod()-1)
            total_return = total_return[-1]
            var = historical_simulation_var(returns, wts, conf_int)
            if var<=target_var and total_return/var >= ret_var:
                final_weights = wts
                final_var = var
                final_total_return = total_return
                ret_var = total_return/var
            p=p+1
            

weights_optimization(returns, max_weights, 0.04, 0.99, 100)            
        